In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time
import os
from numpy import random
import pickle
from playsound import playsound

from bs4 import BeautifulSoup

In [3]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [4]:
if os.name == 'nt':
    driver = webdriver.Chrome('C:/Users/user/Documents/chromedriver.exe')
else:
    driver = webdriver.Chrome('/home/guillermo/Documentos/chromedriver_linux64/chromedriver')

In [4]:
urls= ['https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag={}'.format(i) for i in range(1, 3)]

In [11]:
data = []
metraje = []
url_publicacion = []

for url in urls:
    try:
        driver.get(url)
    except:
        print('Get method failed, check URL.')
    pagina = [e.text.splitlines() for e in driver.find_elements_by_css_selector('.contenedor-info')]
    data.append(pagina)
    html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector('.contenedor-info .mas-info')]
    for i in html:
        soup = BeautifulSoup(i, "html.parser")
        metraje.append(soup.span.text)
        url_publicacion.append(soup.find('a')['href'])
flat_list = flatten(data)
result = pd.DataFrame(flat_list, columns=['desc', 'valor'])
result['metraje'] = metraje
result['metraje'] = result['metraje'].replace({' m2': ''}, regex=True)
result['url'] = url_publicacion

In [13]:
# recupera latlong si hay
coordenadas = []

driver.get('https://www.gallito.com.uy/due%C3%B1o-vende-casa-ph-inmuebles-19426843')

elem = driver.find_elements_by_css_selector('#ubicacion')

if elem != []:
    element = driver.find_element_by_xpath("/html/body/form/main/div/div[1]/ul/li[4]/a/i")
    element.click()
    time.sleep(2)
    ubic_html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector("#ubicacion")]
    soup = BeautifulSoup(ubic_html[0], "html.parser")
    src = soup.find('iframe')['src']
    latlng = src.split('q=', 1)[1].split('0&zoom=', 1)[0]
    coordenadas.append(latlng)
else:
    print('Sin ubicación')
    coordenadas.append('')


Sin ubicación


In [20]:
coordenadas = []

for url in urls:
    driver.get(url)

    elem = driver.find_elements_by_css_selector('#ubicacion')

    if elem != []:
        element = driver.find_element_by_xpath("/html/body/form/main/div/div[1]/ul/li[4]/a/i")
        element.click()
        time.sleep(2)
        ubic_html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector("#ubicacion")]
        soup = BeautifulSoup(ubic_html[0], "html.parser")
        src = soup.find('iframe')['src']
        latlng = src.split('q=', 1)[1].split('&zoom=', 1)[0]
        coordenadas.append(latlng)
    else:
        print('Sin ubicación')
        coordenadas.append('')

Sin ubicación
Sin ubicación


In [25]:
def scrapper(urls):
    data = []
    metraje = []
    url_publicacion=[]

    for url in urls:
        try:
            driver.get(url)
            time.sleep(random.uniform(0.9, 2.3))
        except:
            print('Get method failed, check URL.')
        pagina = [e.text.splitlines() for e in driver.find_elements_by_css_selector('.contenedor-info')]
        data.append(pagina)
        html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector('.contenedor-info .mas-info')]

        for i in html:
            soup = BeautifulSoup(i, "html.parser")
            metraje.append(soup.span.text)
            url_publicacion.append(soup.find('a')['href'])

    flat_list = flatten(data)
    result = pd.DataFrame(flat_list, columns=['desc', 'valor'])
    result['metraje'] = metraje
    result['metraje'] = result['metraje'].replace({' m2': ''}, regex=True)
    result['url'] = url_publicacion

    return result

In [26]:
urls= ['https://www.gallito.com.uy/inmuebles/casas/venta/montevideo/ord_asc?pag={}'.format(i) for i in range(1, 154)]

data = scrapper(urls)

In [27]:
data

,desc,valor,metraje,url
0,3 Dormitorios en Pocitos,U$S 1.250.000,450,https://www.gallito.com.uy/nueva-a-estrenar-al...
1,3 Dormitorios en Carrasco,U$S 1.250.000,354,https://www.gallito.com.uy/casa-carrasco-inmue...
2,4 Dormitorios en Carrasco,U$S 1.250.000,435,https://www.gallito.com.uy/casa-en-venta-inmue...
3,4 Dormitorios en Carrasco,U$S 1.250.000,540,https://www.gallito.com.uy/amplia-residencia-e...
4,4 Dormitorios en Carrasco,U$S 1.250.000,390,https://www.gallito.com.uy/unica-residencia-a-...
...,...,...,...,...
145,Más de 4 dormitorios en Puerto Buceo,U$S 3.000.000,1288,https://www.gallito.com.uy/casi-rbla-puerto-de...
146,1 Ambiente en Cordon,U$S 3.600.000,3600,https://www.gallito.com.uy/gran-terreno-proxim...
147,Más de 4 dormitorios en Pocitos,U$S 4.500.000,1000,https://www.gallito.com.uy/residencia-de-categ...
148,Más de 4 dormitorios en Pocitos,U$S 4.500.000,1200,https://www.gallito.com.uy/casa-unica-con-gran...


In [ ]:
data.shape

In [ ]:
data.to_csv('ventas_mvdeo_20210707.csv', index=False)

## Recupera coordenadas

In [5]:
data = pd.read_csv('ventas_mvdeo_20210707.csv')
data.head()

,desc,valor,metraje,url
0,2 Dormitorios en Ciudad Vieja,U$S 1,160.0,https://www.gallito.com.uy/cerrito-y-juan-carl...
1,Más de 4 dormitorios en Pocitos,U$S 1,300.0,https://www.gallito.com.uy/casa-avenida-brasil...
2,Más de 4 dormitorios en Otro,U$S 100,500.0,https://www.gallito.com.uy/compro-terreno-o-“c...
3,3 Dormitorios en Maroñas Curva,U$S 115,82.0,https://www.gallito.com.uy/casa-muy-cuidada-a-...
4,3 Dormitorios en Goes,U$S 120,67.0,https://www.gallito.com.uy/oportunidad-inversi...


In [6]:
def scrap_latlng(urls):
    coordenadas = []

    for url in urls:
        try:
            driver.get(url)
            time.sleep(1)
        except:
            print('Get method failed, check URLs.')

        elem = driver.find_elements_by_css_selector('#ubicacion')

        if elem != []:
            element = driver.find_element_by_xpath("/html/body/form/main/div/div[1]/ul/li[4]/a/i")
            element.click()
            time.sleep(1)
            ubic_html = [e.get_attribute("innerHTML") for e in driver.find_elements_by_css_selector("#ubicacion")]
            soup = BeautifulSoup(ubic_html[0], "html.parser")
            src = soup.find('iframe')['src']
            latlng = src.split('q=', 1)[1].split('&zoom=', 1)[0]
            if latlng == '/,/':
                coordenadas.append('Nan,Nan')
            else:
                coordenadas.append(latlng)
        else:
            coordenadas.append('Nan,Nan')

    return coordenadas

In [7]:
urls = list(data.url.values)
len(urls)

6110

In [8]:
urls_1 = urls[0:1000]
urls_2 = urls[1000:2000]
urls_3 = urls[2000:3000]
urls_4 = urls[3000:4000]
urls_5 = urls[4000:5000]
urls_6 = urls[5000:6110]

len(urls_1)

1000

In [8]:
# CORRE EL SCRAPPER
coords_1 = scrap_latlng(urls_1)

# guarda en in pickle
with open('coords_1.pkl', 'wb') as f:
    pickle.dump(coords_1, f)

In [17]:
# CORRE EL SCRAPPER
coords_2 = scrap_latlng(urls_2)

with open('coords_2.pkl', 'wb') as f:
    pickle.dump(coords_2, f)

In [9]:
# CORRE EL SCRAPPER
coords_3 = scrap_latlng(urls_3)

with open('coords_3.pkl', 'wb') as f:
    pickle.dump(coords_3, f)

playsound('ALARM.WAV')

In [ ]:
# CORRE EL SCRAPPER
coords_4 = scrap_latlng(urls_4)

with open('coords_4.pkl', 'wb') as f:
    pickle.dump(coords_4, f)

playsound('ALARM.WAV')

In [ ]:
# CORRE EL SCRAPPER
coords_5 = scrap_latlng(urls_5)

with open('coords_5.pkl', 'wb') as f:
    pickle.dump(coords_5, f)

In [ ]:
# CORRE EL SCRAPPER
coords_6 = scrap_latlng(urls_6)

with open('coords_6.pkl', 'wb') as f:
    pickle.dump(coords_6, f)

In [10]:
coords_3

38',
 'Nan,Nan',
 'Nan,Nan',
 'Nan,Nan',
 'NAN,NAN',
 '-34.902316,-56.130631',
 '-34.9012137,-56.130135',
 '-34.910193638135375,-56.1620072458191',
 '-34.878573,-56.157109',
 '-34.87896295051338,-56.15756550769042',
 '-34.877818743875885,-56.15627804736327',
 'Nan,Nan',
 '-34.8859497,-56.17101719999999',
 '-34.92517997437255,-56.15618148783874',
 'Nan,Nan',
 '-34.84239327192973,-56.20657483081054',
 '-34.8636188,-56.1663296',
 '-34.9086868,-56.1607388',
 '-34.89161857365982,-56.16327324847411',
 '-34.8871623,-56.1540975',
 '-34.899556799999999,-56.148286699999971',
 '-34.8482857,-56.2071118',
 '-34.85493126667175,-56.158466729919425',
 'Nan,Nan',
 '-34.800131,-56.249400',
 'Nan,Nan',
 '-34.8974556814384100,-56.1670747412030660',
 '-34.82933940615807,-56.21208796137694',
 '-34.88563500654022,-56.13441167301636',
 '-34.872799,-56.198373',
 'Nan,Nan',
 '-34.88979691767839,-56.09491983393859',
 'Nan,Nan',
 '-34.892014,-56.144231',
 '-34.897989651769606,-56.144991311828605',
 '-34.889594508

In [36]:
data[['lat', 'lng']] = [i.split(',') for i in coords]

In [42]:
data['lat'] = data['lat'].astype(float).round(5)
data['lng'] = data['lng'].astype(float).round(5)

In [44]:
data.head(8)

,desc,valor,metraje,url,lat,lng
0,2 Dormitorios en Ciudad Vieja,U$S 1,160,https://www.gallito.com.uy/cerrito-y-juan-carl...,-34.90460,-56.20420
1,Más de 4 dormitorios en Pocitos,U$S 1,300,https://www.gallito.com.uy/casa-avenida-brasil...,-34.90719,-56.15927
2,Más de 4 dormitorios en Otro,U$S 100,500,https://www.gallito.com.uy/compro-terreno-o-“c...,-34.90721,-56.19027
3,3 Dormitorios en Maroñas Curva,U$S 115,82,https://www.gallito.com.uy/casa-muy-cuidada-a-...,-34.86014,-56.13203
4,3 Dormitorios en Goes,U$S 120,67,https://www.gallito.com.uy/oportunidad-inversi...,-34.90721,-56.19027
5,2 Dormitorios en Prado,U$S 199,,https://www.gallito.com.uy/dueño-vende-casa-ph...,NaN,NaN
6,2 Dormitorios en Maroñas,$U 16.500,59,https://www.gallito.com.uy/2-dormitorios-2-bañ...,NaN,NaN
7,1 Dormitorio en Colon,$U 65.000,50,https://www.gallito.com.uy/muy-linda-casa-en-b...,-34.81525,-56.21593


In [45]:
data.to_csv('ventas_mvdeo_20210707_con_coord.csv', index=False)

In [ ]:
# limpieza
data = pd.read_csv('ventas_mvdeo_20210706.csv')

data['valor'].str.replace('U', '')

for word, rep in {"U":" ", "S":"", "$":"", ".":""}.items():
    data['valor'] = data['valor'].str.replace(word, rep, regex=False)

In [ ]:
data['valor'] = data['valor'].astype(int)

In [ ]:
len(data)

In [ ]:
data = data.loc[~((data.valor == 111111111) | (data.valor < 45000))]
data.head()

In [ ]:
data[['dormitorios', 'barrio']] = data['desc'].str.split(' en ', n = 1, expand = True)

In [ ]:
# data.to_csv('ventas_mvdeo_20210706_depu.csv', index=False)

In [ ]:
# pd.Series(data.barrio.unique()).to_csv('cod.csv')

In [ ]:
data['barrio_ine'] = data['barrio']

In [ ]:
def format_barrio(df, column):
    "Formatea strings de Barrio Gallito a Barrio INE"
    barrios_dict = {
        'Aguada' : 'Aguada',
        'Aires Puros' : 'Aires Puros',
        'Atahualpa' : 'Atahualpa',
        'B. De Carrasco' : 'Bañados de Carrasco',
        'Barrio Sur' : 'Barrio Sur',
        'Belvedere' : 'Belvedere',
        'Brazo Oriental' : 'Brazo Oriental',
        'Buceo' : 'Buceo',
        'Capurro' : 'Capurro, Bella Vista',
        'Carrasco' : 'Carrasco',
        'Carrasco Norte' : 'Carrasco Norte',
        'Casabo' : 'Casabó, Pajas Blancas',
        'Casavalle' : 'Casavalle',
        'Centro' : 'Centro',
        'Cerrito' : 'Cerrito',
        'Cerro' : 'Cerro',
        'Ciudad Vieja' : 'Ciudad Vieja',
        'Cno. Maldonado' : 'Colón Centro y Noroeste',
        'Colon' : 'Colón Sureste, Abayubá',
        'Cordon' : 'Cordón',
        'Golf' : 'Flor de Maroñas',
        'Ituzaingo' : 'Ituzaingó',
        'J. Hipodromo' : 'Jardines del Hipódromo',
        'Jacinto Vera' : 'Jacinto Vera',
        'La Blanqueada' : 'La Blanqueada',
        'La Comercial' : 'La Comercial',
        'La Figurita' : 'La Figurita',
        'La Teja' : 'La Teja',
        'Larrañaga' : 'Larrañaga',
        'Las Acacias' : 'Las Acacias',
        'Lezica' : 'Lezica, Melilla',
        'Malvin' : 'Malvín',
        'Malvin Norte' : 'Malvín Norte',
        'Manga' : 'Manga',
        'Maroñas' : 'Maroñas, Parque Guaraní',
        'Maroñas Curva' : 'Maroñas, Parque Guaraní',
        'Melilla' : 'Lezica, Melilla',
        'Nuevo Paris' : 'Mercado Modelo, Bolívar',
        'Pajas Blancas' : 'Nuevo París',
        'Palermo' : 'Palermo',
        'Parque Batlle' : 'Parque Batlle, Villa Dolores',
        'Parque Rodo' : 'Parque Rodó',
        'Paso De La Arena' : 'Paso de la Arena',
        'Peñarol' : 'Peñarol, Lavalleja',
        'Perez Castellanos' : 'Castro, P. Castellanos',
        'Piedras Blancas' : 'Piedras Blancas',
        'Pocitos' : 'Pocitos',
        'Pocitos Nuevo' : 'Pocitos',
        'Prado' : 'Prado, Nueva Savona',
        'Prado Norte' : 'Prado, Nueva Savona',
        'Puerto Buceo' : 'Buceo',
        'Punta Carretas' : 'Punta Carretas',
        'Punta Gorda' : 'Punta Gorda',
        'Punta Rieles' : 'Punta Rieles, Bella Italia',
        'Reducto' : 'Reducto',
        'Sayago' : 'Sayago',
        'Tres Cruces' : 'Tres Cruces',
        'Union' : 'Unión',
        'Villa Dolores' : 'Parque Batlle, Villa Dolores',
        'Villa Española' : 'Villa Española',
        'Villa Muñoz' : 'Villa Muñoz, Retiro'
        }

    return df[column].map(barrios_dict)

In [ ]:
data['barrio_ine'] = format_barrio(data, 'barrio')

In [ ]:
data.head(15)

In [ ]:
def encode_barrio(df, column):
    "Codifica barrio INE"
    barrios_dict = {
    'Ciudad Vieja':1,
    'Centro':2,
    'Barrio Sur':3,
    'Cordón':4,
    'Palermo':5,
    'Parque Rodó':6,
    'Punta Carretas':7,
    'Pocitos':8,
    'Buceo':9,
    'Parque Batlle, Villa Dolores':10,
    'Malvín':11,
    'Malvín Norte':12,
    'Punta Gorda':13,
    'Carrasco':14,
    'Carrasco Norte':15,
    'Bañados de Carrasco':16,
    'Maroñas, Parque Guaraní':17,
    'Flor de Maroñas':18,
    'Las Canteras':19,
    'Punta Rieles, Bella Italia':20,
    'Jardines del Hipódromo':21,
    'Ituzaingó':22,
    'Unión':23,
    'Villa Española':24,
    'Mercado Modelo, Bolívar':25,
    'Castro, P. Castellanos':26,
    'Cerrito':27,
    'Las Acacias':28,
    'Aires Puros':29,
    'Casavalle':30,
    'Piedras Blancas':31,
    'Manga, Toledo Chico':32,
    'Paso de las Duranas':33,
    'Peñarol, Lavalleja':34,
    'Cerro':35,
    'Casabó, Pajas Blancas':36,
    'La Paloma, Tomkinson':37,
    'La Teja':38,
    'Prado, Nueva Savona':39,
    'Capurro, Bella Vista':40,
    'Aguada':41,
    'Reducto':42,
    'Atahualpa':43,
    'Jacinto Vera':44,
    'La Figurita':45,
    'Larrañaga':46,
    'La Blanqueada':47,
    'Villa Muñoz, Retiro':48,
    'La Comercial':49,
    'Tres Cruces':50,
    'Brazo Oriental':51,
    'Sayago':52,
    'Conciliación':53,
    'Belvedere':54,
    'Nuevo París':55,
    'Tres Ombúes, Victoria':56,
    'Paso de la Arena':57,
    'Colón Sureste, Abayubá':58,
    'Colón Centro y Noroeste':59,
    'Lezica, Melilla':60,
    'Villa García, Manga Rural':61,
    'Manga':62
        }
    return df[column].map(barrios_dict)

In [ ]:
data['cod_barrio_ine'] = encode_barrio(data, 'barrio_ine')
data.head()

In [ ]:

data = data.loc[(data.metraje > 10) & (data.metraje < 4000) & (~data.metraje.isna())]

data['metraje'] = data['metraje'].astype(int)

data['valor_metro'] = data['valor'] / data['metraje']

agru = data.groupby('cod_barrio_ine').median('valor_metro').reset_index()

agru['cod_barrio_ine'] = agru['cod_barrio_ine'].astype(int)
                                                       
agru.to_csv('agru.csv', index=False)